## File for locate bruise
##### Tresholds, og alt det der

In [ ]:
import numpy as np
import spectral as sp
import matplotlib.pyplot as plt
import cv2
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from functions import *

In [ ]:
names = np.array(["before-hit", "0m","30m","1h" ,"3h","12h", "24h", "48h"])

trainName = "0m"

strawberryCube = sp.open_image(getImage(trainName))
        
        # 0m: [420,5212,666] 30m: [465 ,5189, 666], 1h: [436,5304,666] , 3h: id2 [376, 5292,666],  
        
strawberryCoords = np.array([[420 ,5212, 666]])#
distinctBands = [75,106,175]                                   
strawberriesDistinct = strawberryCube.read_bands( distinctBands ) # Only distinct bandsareaBruised

In [ ]:



strawberries = []  # List of whole strawberries
for coord in strawberryCoords:
    strawberries.append(getRegion(strawberriesDistinct, coord[0], coord[1], coord[2]))

strawberries = np.array(strawberries)

for strawberry in strawberries:    
    sp.imshow(strawberry[:,:,strawberry[-1].shape[-1]-1], title="Strawberry training on")



    # center of bruised area with given radius on different strawberries, index 3 is the radius
                            # id 1             # id 7
bruises     = np.array([[255, 395, 200]])#, [500,500,200]]) #[470, 410, 200], [220, 340, 200]]), 3h: id2[250, 375], id8, [300,350, 200]
bruisetrain = np.array([[225, 395, 150]])#, [500,500,100]]) # [470, 410, 70], [220, 340, 30]]) 



X_train = []
y_train = []

for i in range(len(bruisetrain)):

    areaBruised = getRegion(strawberries[i], bruisetrain[i][0], bruisetrain[i][1], bruisetrain[i][2]) # subimage of a bruised area
    sp.imshow(areaBruised, title="Bruised train data")
    noSeeds = removeSeeds(areaBruised)
    n,bands = noSeeds.shape
    X_train.extend(noSeeds)
    print("Training data bruised spectrums: %d" % (n))
    y_train.extend(np.ones(n))
    
    
    # removes the bruised part on the whole strawberry
for i in range(len(strawberries)):
    strawberries[i][bruises[i][1] - bruises[i][2]//2 : bruises[i][1] + bruises[i][2]//2 , 
                    bruises[i][0] - bruises[i][2]//2 : bruises[i][0] + bruises[i][2]//2] = 0



for strawberry in strawberries:
    
    mask = strawberryMask(strawberry)
    sp.imshow(mask, title="mask")
    values = strawberry[np.where(mask==1)]
    n,bands = values.shape
    
    X_train.extend(values)
    y_train.extend(np.zeros(n))
    
    print("Training data healthy spectrums: %d" %(n))


X_train = np.array(X_train)


X_train = np.array(X_train)
y_train = np.array(y_train)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
from joblib import dump, load

svm = svm.SVC() 
svm.fit(X_train, y_train)
print("SVM trained")

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print("KNN trained")

 # Saving model for later use 

nr = "2" # strawberry id used for training
fileNameSVM = "SVM_" + trainName + "_STB" + nr + ".joblib"
fileNameKNN = "KNN_" + trainName + "_STB" + nr + ".joblib"
dump(svm, fileNameSVM)   # saves model 
dump(knn, fileNameKNN) 

In [ ]:
testName = "3h"

strawberryCube = sp.open_image(getImage(testName))
strawberriesDistinct = strawberryCube.read_bands( distinctBands )[4500:,900:,:] # Only distinct bands and removes the reference obj.

                                             #12h    # berry 1             berry 2          berry 3       berry 1,2,3,4      berry 7,8,9,10
#testimg = getRegion(strawberriesDistinct, 900, 4504, 1800)  #[1224, 5090, 600], [333 ,5060, 666] , [344, 3912] , [900, 4504, 1800], [900, 1960, 1800]
testimg = strawberriesDistinct
sp.imshow(testimg[:,:,testimg.shape[-1]-1], title="Test image " + testName)

mask = strawberryMask(testimg)
sp.imshow(mask, title="Strawberry mask")

x_test = testimg[np.where(mask==1)]

print("Data reduced by not including zeros: %f" % ( 1-np.prod(x_test.shape)/np.prod(testimg.shape) ) )



In [ ]:

print("SVM starting prediction")
predSVM = svm.predict(x_test)
predKNN = knn.predict(x_test)

predictionSVM = mask.copy()
predictionSVM[predictionSVM == 1] = predSVM

print("KNN starting prediction")
predictionKNN = mask.copy()
predictionKNN[predictionKNN == 1] = predKNN


viewSVM = sp.imshow(testimg[:,:,testimg.shape[-1]-1], figsize=(6,6), title="SVM "+trainName+" training on STB 2."+testName+" test results")
viewSVM.set_classes(predictionSVM)
viewSVM.set_display_mode('overlay')
viewSVM.class_alpha = 0.6 #set transparency
viewSVM.show_data

viewKNN = sp.imshow(testimg[:,:,testimg.shape[-1]-1], figsize=(6,6), title="KNN "+trainName+" training on STB 2."+testName+" test results")
viewKNN.set_classes(predictionKNN)
viewKNN.set_display_mode('overlay')
viewKNN.class_alpha = 0.6 #set transparency
viewKNN.show_data

In [ ]:
names = np.array(["before-hit", "0m","30m","1h" ,"3h","12h", "24h", "48h"])

strawberryCube = sp.open_image(getImage("12h"))
strawberryCubeBefore = sp.open_image(getImage("before-hit"))
N,M,Bands = strawberryCube.shape
strawberry = strawberryCube.read_subregion((4700,N), (0,666)) # strawberry ID 1 (750,M)
                                                            # strawberry ID 2 (0,666)
sp.imshow(strawberry[:,:,180])

bruise = getRegion(strawberry, 225, 400, 3) # 
print(bruise.shape)


test8bit = (bruise[:,:,180]*255).astype('uint8')
thresh = cv2.threshold(test8bit, np.mean(test8bit),1, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1] # inverse otsu threshold
sp.imshow(bruise[:,:,180], cmap="gray")

a = np.where(thresh==1)

X_train = bruise[a]
print(X_train.shape)




In [ ]:
names = np.array(["before-hit", "0m","30m","1h" ,"3h","12h", "24h", "48h"])

strawberryCube = sp.open_image(getImage("12h"))
N,M,Bands = strawberryCube.shape
strawberry = strawberryCube.read_subregion((4700,N), (0,666)) # strawberry ID 1 (750,M)
                                                              # strawberry ID 2 (0,666)

sp.imshow(strawberry[:,:,180],figsize=(5,10))



test = strawberry[:,:,180]
test8bit = (test*255).astype('uint8')
#th3 = cv2.adaptiveThreshold(test8bit,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,21,1)
tresh = cv2.threshold(test8bit,10,255, cv2.THRESH_BINARY)[1]
kernel = np.ones((7,7),np.uint8)
closing  = cv2.morphologyEx(tresh, cv2.MORPH_CLOSE, kernel, iterations=1)
erode = cv2.erode(closing,kernel, iterations =10)

#mask = erode
kernel = np.ones((3,3),np.uint8)

imggrad =  (cv2.dilate(test,kernel) - cv2.erode(test,kernel)) #* mask

imggrad[imggrad < imggrad[300,210]] = 0
imggrad[imggrad >= imggrad[300,210]] = 1
eroded = cv2.dilate(cv2.erode(imggrad,kernel, iterations =5),kernel,iterations =2)

sp.imshow(eroded,figsize=(5,10))

imgNoSeeds = test*(1-eroded)

sp.imshow(imgNoSeeds,figsize=(5,10))


